In [44]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score , classification_report , multilabel_confusion_matrix
import cv2
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.utils import to_categorical

import os
import warnings
warnings.filterwarnings('ignore')
import sys
import json

In [ ]:
def splitting_data(dest_path , req_images) :
  try :
    import os
    import random
    import shutil
    # Define the source and destination directories
    source_directory = '/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/InternShip'  # Modify with the path to your source folder
    destination_directory = dest_path  # Modify with the path to your destination folder
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
    # Loop through each folder in the source directory
    for folder_name in os.listdir(source_directory):
        folder_path = os.path.join(source_directory, folder_name)
        # Check if it's a directory (not a file)
        if os.path.isdir(folder_path):
            # Create a new folder in the destination directory with the same name
            new_folder_path = os.path.join(destination_directory, folder_name)
            if not os.path.exists(new_folder_path):
                os.makedirs(new_folder_path)
            # Get all images in the current folder
            images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
            # Select 200 random images (or fewer if there are less than 200)
            selected_images = random.sample(images, min(req_images, len(images)))
            # Copy the selected images to the new folder in the destination directory
            for image in selected_images:
                source_image = os.path.join(folder_path, image)
                destination_image = os.path.join(new_folder_path, image)
                shutil.copy(source_image, destination_image)
            print(f"Processed folder '{folder_name}': {len(selected_images)} images copied.")
  except Exception as e :
    er_type, er_msg, line_no = sys.exc_info()
    print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')

In [ ]:
# splitting_data("/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Train_data" , 150)

In [ ]:
# splitting_data("/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Test_data" , 20)

In [ ]:
# splitting_data("/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Validation_data" , 30)

In [8]:
class CustomModel :
  def __init__(self, train_path , test_path , validation_path , model_path) :
    try :
      self.train_path = train_path
      self.test_path = test_path
      self.validation_path = validation_path
      self.target_labels = os.listdir(self.train_path)
      self.model = Sequential()
      self.model_path = model_path
    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')

  def ImageProcessing(self) :
    try :
      train_data_process = ImageDataGenerator(rescale=1/255 ,
                                  shear_range = 0.2 ,
                                  zoom_range = 0.2 ,
                                  horizontal_flip = True)
      test_data_process = ImageDataGenerator(rescale=1/255)
      val_data_process = ImageDataGenerator(rescale=1/255)

      self.train_data = train_data_process.flow_from_directory(self.train_path ,
                                                      target_size = (256,256),
                                                      classes = self.target_labels ,
                                                      class_mode = "categorical",
                                                      batch_size = 16)
      self.test_data = test_data_process.flow_from_directory(self.test_path,
                                                          classes = self.target_labels,
                                                          target_size=(256,256))

      self.validation_data = val_data_process.flow_from_directory(self.validation_path ,
                                                                  classes = self.target_labels,
                                                                  target_size=(256,256))
    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')

  def Model(self) :
    try :
      self.model.add(Conv2D(128 , kernel_size = (3,3) , input_shape = (int(256),int(256),3) , padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))
      # second kernel and max poo layers
      self.model.add(Conv2D(64 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))
      # third kernel and max poo layers
      self.model.add(Conv2D(32 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))
      # forth kernel and max poo layers
      self.model.add(Conv2D(6 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))
      self.model.add(Flatten()) # one dimensional array
      # above 1d data give to ANN
      self.model.add(Dense(32,activation='relu')) # hiddel layer 1
      self.model.add(Dense(16,activation = 'relu')) # hiddel layer 2

      # output layer
      self.model.add(Dense(len(self.target_labels),activation='softmax'))
      self.model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['Accuracy'])
    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')

  def Training(self) :
    try :
      self.Model()
      self.ImageProcessing()
      self.model.fit(self.train_data,
                    validation_data = self.validation_data,
                    epochs=15)

      self.model.save(f'{self.model_path}/custom_model.h5')

    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')


In [ ]:
if __name__ == "__main__" :
  train_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Train_data"
  test_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Test_data"
  validation_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Validation_data"
  model_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Model"

  c_Model = CustomModel(train_path , test_path , validation_path , model_path)
  c_Model.Training()

Found 5094 images belonging to 34 classes.
Found 680 images belonging to 34 classes.
Found 1020 images belonging to 34 classes.
Epoch 1/15
319/319 ━━━━━━━━━━━━━━━━━━━━ 2569s 8s/step - Accuracy: 0.0301 - loss: 3.5268 - val_Accuracy: 0.0235 - val_loss: 3.5263
Epoch 2/15
319/319 ━━━━━━━━━━━━━━━━━━━━ 2124s 7s/step - Accuracy: 0.0272 - loss: 3.5257 - val_Accuracy: 0.0294 - val_loss: 3.5264
Epoch 3/15
319/319 ━━━━━━━━━━━━━━━━━━━━ 2119s 7s/step - Accuracy: 0.0288 - loss: 3.5267 - val_Accuracy: 0.0304 - val_loss: 3.5264
Epoch 4/15
319/319 ━━━━━━━━━━━━━━━━━━━━ 2136s 7s/step - Accuracy: 0.0257 - loss: 3.5267 - val_Accuracy: 0.0294 - val_loss: 3.5264
Epoch 5/15
319/319 ━━━━━━━━━━━━━━━━━━━━ 2103s 7s/step - Accuracy: 0.0240 - loss: 3.5270 - val_Accuracy: 0.0294 - val_loss: 3.5264
Epoch 6/15
319/319 ━━━━━━━━━━━━━━━━━━━━ 2106s 7s/step - Accuracy: 0.0220 - loss: 3.5267 - val_Accuracy: 0.0294 - val_loss: 3.5264
Epoch 7/15
319/319 ━━━━━━━━━━━━━━━━━━━━ 2109s 7s/step - Accuracy: 0.0272 - loss: 3.5268 - va

In [42]:
class Evaluation :
  def __init__(self , test_path , model) :
    try :
      self.model = model
      self.test_path = test_path
      self.target_labels = sorted(os.listdir(self.test_path))
      test_data_process = ImageDataGenerator(rescale=1/255)
      self.test_data = test_data_process.flow_from_directory(self.test_path,
                                                          classes = self.target_labels,
                                                          target_size=(256,256))
    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')
  def Validation(self , report_path) :
    try:
      y_true = np.array(self.test_data.labels)
      y_true = to_categorical(y_true, num_classes=len(self.target_labels))
      y_pred_prob = self.model.predict(self.test_data)  # Get probability scores
      y_pred = (y_pred_prob > 0.5).astype(int)
      conf_matrices = multilabel_confusion_matrix(y_true, y_pred)

      # Summing over all labels to get overall TP, TN, FP, FN
      TP = conf_matrices[:, 1, 1].sum()
      TN = conf_matrices[:, 0, 0].sum()
      FP = conf_matrices[:, 0, 1].sum()
      FN = conf_matrices[:, 1, 0].sum()

      accuracy = (TP + TN) / (TP + TN + FP + FN)
      precision = TP/(TP+FP) if (TP + FP) > 0 else 0
      recall = TP / (TP+FN) if (TP + FN) > 0 else 0
      f1_sc = 2 * ((precision*recall) / (precision + recall)) if (precision + recall) > 0 else 0

      report = {"True Positives" : TP,
                "True Negatives" : TN,
                "Flase Positives" : FP,
                "False Negatives" : FN ,
                "Accuracy" : accuracy ,
                "Precision" : precision ,
                "Recall" : recall,
                "F1_Score" : f1_sc
                }

      def convert_numpy(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert arrays to lists
        elif isinstance(obj, (np.int64, np.int32)):
            return int(obj)  # Convert NumPy integers to Python int
        elif isinstance(obj, (np.float64, np.float32)):
            return float(obj)  # Convert NumPy floats to Python float
        else:
            return obj

      with open(report_path, "w") as json_file:
        json.dump(report, json_file, indent=4 , default=convert_numpy)


      print(f"Classification report saved successfully")

    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')


In [43]:
model = tensorflow.keras.models.load_model("/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Model/custom_model.h5")
test_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Test_data"
obj = Evaluation(test_path , model)
obj.Validation("/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Model/validation_report.json")

Found 680 images belonging to 34 classes.
22/22 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step
Classification report saved successfully
